In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Conv2D, ConvLSTM2D, Conv3D, MaxPooling2D, Dropout, \
    MaxPooling3D
from keras.layers.normalization import BatchNormalization
from keras.models import Model, Sequential
from keras.utils import plot_model
import json

from EmoPy.src.callback import PlotLosses

/Users/tianchenwang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [29]:
class _FERNeuralNet(object):
    """
    Interface for all FER deep neural net classes.
    """

    def __init__(self, emotion_map):
        self.emotion_map = emotion_map
        self._init_model()

    def _init_model(self):
        raise NotImplementedError("Class %s doesn't implement _init_model()" % self.__class__.__name__)

    def fit(self, x_train, y_train):
        raise NotImplementedError("Class %s doesn't implement fit()" % self.__class__.__name__)

    def fit_generator(self, generator, validation_data=None, epochs=50):
        self.model.compile(optimizer="RMSProp", loss="cosine_proximity", metrics=["accuracy"])
        self.model.fit_generator(generator=generator, validation_data=validation_data, epochs=epochs,
                                 callbacks=[ReduceLROnPlateau(), EarlyStopping(patience=3), PlotLosses()])

    def predict(self, images):
        self.model.predict(images)

    def save_model_graph(self):
        plot_model(self.model, to_file='output/model.png')

    def export_model(self, model_filepath, weights_filepath, emotion_map_filepath, emotion_map):
        self.model.save_weights(weights_filepath)

        model_json_string = self.model.to_json()
        model_json_file = open(model_filepath, 'w')
        model_json_file.write(model_json_string)
        model_json_file.close()

        with open(emotion_map_filepath, 'w') as fp:
            json.dump(emotion_map, fp)

In [30]:
class ConvolutionalNN(_FERNeuralNet):
    """
    2D Convolutional Neural Network
    :param image_size: dimensions of input images
    :param channels: number of image channels
    :param emotion_map: dict of target emotion label keys with int values corresponding to the index of the emotion probability in the prediction output array
    :param filters: number of filters/nodes per layer in CNN
    :param kernel_size: size of sliding window for each layer of CNN
    :param activation: name of activation function for CNN
    :param verbose: if true, will print out extra process information
    **Example**::
        net = ConvolutionalNN(target_dimensions=(64,64), channels=1, target_labels=[0,1,2,3,4,5,6], time_delay=3)
        net.fit(features, labels, validation_split=0.15)
    """

    def __init__(self, image_size, channels, emotion_map, filters=10, kernel_size=(4, 4), activation='relu',
                 verbose=False):
        #we define what kind of self we want, image that self = person, and image_size can be name, channels can be age```
        self.channels = channels
        self.image_size = image_size
        self.verbose = verbose

        self.filters = filters
        self.kernel_size = kernel_size
        self.activation = activation
        super().__init__(emotion_map)

    def _init_model(self):
        """
        Composes all layers of 2D CNN.
        """
        model = Sequential()
        # Sequential() is a model in Keras which is a linear stack of layers
        model.add(Conv2D(input_shape=list(self.image_size) + [self.channels], filters=self.filters,
                         kernel_size=self.kernel_size, activation='relu', data_format='channels_last'))
        #input指的是需要做卷积的输入图像，其具体含义是图片高度 图片宽度 图像管道数
        #filter 为卷积核，具体含义是[卷积核的高度，卷积核的宽度，图像通道数，卷积核个数]，要求类型与参数input相同
        # specifying the width and height of the 2D convolution window. K kernels waiting to be applied to the image, each kernel is convolved with input volumn and the output of each convolution 
        #operation produces a 2D output,called activation map
        #activation 函数 为relu，增加non-linear性
        #channels last = 输入个个维度的顺序，channel lasy对应：batch，steps，channels
        #This layer creates a convolution kernel that is convolved with the layer input to produce a tensor of outputs.
        #The image shows the differences in size of each of the network layers.
        #Convolutional layers apply a convolution operation to the input, passing the result to the next layer
        #The convolution emulates the response of an individual neuron to visual stimuli.
        #Pooling layers are used to reduce the input space and thus complexity and processing time.
        #The flattening layer converts the output of the previous layer to a one-dimensional vector,
        #and the final layer takes that vector and calculates a final classification output.
        model.add(
            Conv2D(filters=self.filters, kernel_size=self.kernel_size, activation='relu', data_format='channels_last'))
        model.add(MaxPooling2D())
        model.add(
            Conv2D(filters=self.filters, kernel_size=self.kernel_size, activation='relu', data_format='channels_last'))
        model.add(
            Conv2D(filters=self.filters, kernel_size=self.kernel_size, activation='relu', data_format='channels_last'))
        model.add(MaxPooling2D())

        model.add(Flatten())
        model.add(Dense(units=len(self.emotion_map.keys()), activation="relu"))
        if self.verbose:
            model.summary()
        self.model = model

    def fit(self, image_data, labels, validation_split, epochs=50):
        """
        Trains the neural net on the data provided.
        :param image_data: Numpy array of training data.
        :param labels: Numpy array of target (label) data.
        :param validation_split: Float between 0 and 1. Percentage of training data to use for validation
        :param batch_size:
        :param epochs: number of times to train over input dataset.
        """
        self.model.compile(optimizer="RMSProp", loss="cosine_proximity", metrics=["accuracy"])
        self.model.fit(image_data, labels, epochs=epochs, validation_split=validation_split,
                       callbacks=[ReduceLROnPlateau(), EarlyStopping(patience=3)])

In [3]:
from EmoPy.src.fermodel import FERModel
from EmoPy.src.directory_data_loader import DirectoryDataLoader
from EmoPy.src.data_generator import DataGenerator
from pkg_resources import resource_filename,resource_exists
import pandas as pd
import numpy as np
import os
from EmoPy.src.csv_data_loader import CSVDataLoader
import cv2
import time
from EmoPy.src.neuralnets import ConvolutionalNN

In [6]:
validation_split = 0.15

target_dimensions = (64, 64)
channels = 1
verbose = True

print('--------------- Convolutional Model -------------------')
print('Loading data...')
# 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
raw_dimensions = (48, 48)
target_dimensions = (64, 64)
verbose = True
channels = 3

emotion_map = {'0':'Angry',
              '1':'Disgust',
              '2':'Fear',
              '3':'Happy',
              '4':'Sad',
              '5':'Surprise',
              '6':'Neutral'}
data_loader = CSVDataLoader(emotion_map, datapath='../data/fer2013.csv',
                            image_dimensions=raw_dimensions, 
                            csv_label_col=0, csv_image_col=1, out_channels=3)

dataset = data_loader.load_data()

if verbose:
    dataset.print_data_details()

print('Preparing training/testing data...')
train_images, train_labels = dataset.get_training_data()
train_gen = DataGenerator().fit(train_images, train_labels)
test_images, test_labels = dataset.get_test_data()
test_gen = DataGenerator().fit(test_images, test_labels)

print('Training net...')
model = ConvolutionalNN(target_dimensions, channels, dataset.get_emotion_index_map(), 
                        filters = 50, kernel_size=(2,2), verbose=True)
model.fit_generator(train_gen.generate(target_dimensions, batch_size=5),
                    test_gen.generate(target_dimensions, batch_size=5),
                    epochs=5)

# Save model configuration
# model.export_model('output/conv2d_model.json','output/conv2d_weights.h5',"output/conv2d_emotion_map.json", emotion_map)

--------------- Convolutional Model -------------------
Loading data...
Extracting training data from csv...

DATASET DETAILS
35887 image samples
28709 training samples
7178 test samples

Preparing training/testing data...
Training net...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 63, 63, 50)        650       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 62, 62, 50)        10050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 31, 25)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 61, 30, 50)        5050      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 60, 29, 50)        10050     
___________________________________

KeyboardInterrupt: 

In [ ]:
# # training model CNNLSTM
# from EmoPy.src.neuralnets import ConvolutionalLstmNN

# start = time.time()
# model = ConvolutionalLstmNN(target_dimensions, channels, 
#                             dataset.get_emotion_index_map(), verbose=True)

# model.fit_generator(train_gen.generate(target_dimensions, batch_size=5),
#                     test_gen.generate(target_dimensions, batch_size=5),
#                     epochs=5)

# end = time.time()
# print(end-start)